In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import time
import random
import re
from tqdm import tqdm
# from selenium import webdriver
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import time
import requests
import sys
import json
import pickle


# 크롤링 된 결과 확인 및 저장

In [3]:
import pickle

In [2]:
path = 'C:/Users/82103/text_data/'

In [9]:
# # 저장
# with open(path+'naver_dict.pickle', 'wb') as f:
#     pickle.dump(naver_dict, f)

# with open(path+'daum_dict.pickle', 'wb') as f:
#     pickle.dump(daum_dict, f)

In [4]:
# 읽기
with open(path+'naver_dict.pickle', 'rb') as f:
    naver_dict = pickle.load(f)
    
with open(path+'daum_dict.pickle', 'rb') as f:
    daum_dict = pickle.load(f)

In [5]:
len(naver_dict['titles'])

1896

In [6]:
len(daum_dict['titles'])

276

# 자연어 전처리

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt, Mecab
import re
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

m=Mecab('C:/mecab/mecab-ko-dic')
o = Okt()

In [3]:
# 속도 향상을 위해 gpu 켜기
import torch
torch.cuda.is_available()

True

In [4]:
# 단어 분리할 한국어 모델 학습
corpus = DoublespaceLineCorpus("soynlp_practice_final.txt")
len(corpus)

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)

training was done. used memory 1.249 Gbmory 0.326 Gb
all cohesion probabilities was computed. # words = 298191
all branching entropies was computed # words = 500472
all accessor variety was computed # words = 500472


In [5]:
# 불용어 사전 불러오기 
none_dic = pd.read_csv('./none_dic.csv', header = None, encoding = "cp949")
none = none_dic[0].tolist()
stopwords = none

# 토큰화할 방법 지정 (mecab, Okt())
tokenizer = m

In [6]:
# 내용 관련 필터링 함수 생성
def content_clean(dic):
    
    content = dic['contents']
    clean_list = []
    p = re.compile("\W+|\d+")
    for i in range(len(content)):
        try:
            # 1. 특수문자, 숫자 삭제
            if len(content[i]) == 1:
                sent = p.sub(" ", content[i][0]) 
            else:
                sent = p.sub(" ", content[i][1]) 
                
            # 2. konlpy에서 형태소 분석을 통해 형태소로 분리
            tokens = []
            token = tokenizer.morphs(sent) # 모든 형태소 처리(morphs) ,명사만 뽑기(nouns)
            token = [t for t in token if t not in stopwords] # 불용어 필터
            token = [t for t in token if len(t) > 1] # 글자 하나는 제거
            tokens.append(' '.join(token))

            # 3. L tokenizer 이용해서 토큰 리스트 생성
            clean = []
            words = l_tokenizer.tokenize(tokens[0], flatten = False)
            words = [x for x, y in words if len(x) > 1] # 앞에꺼(L)만 출력, 두 글자 이상만
            words = [x for x in words if x not in stopwords] # 불용어 제거해주기
            clean.append(' '.join(words))

        except:
            pass
        clean_list.append(clean)
        
    return(clean_list)

In [313]:
content_naver = content_clean(naver_dict)

In [314]:
content_daum = content_clean(daum_dict)

In [317]:
print(len(content_naver), len(content_daum))

1896 276


In [7]:
naver_dict['comments']

[[['"다른건 몰라도 공유킥보드는 없애는게 맞다 무면허에 노헬멧에 짐승들 한쌍이 타는 경우도 많아서 상당히 위험하다"',
   '"공유킥보드는 안전도 안전이지만 생각보다 요금 쎔. 가끔 신호 잘못 걸리면 그냥 편안한 택시가 맞는듯 이라는 생각도 들고 적절과 애매 사이에서 여러 생각을 하게 만드는 이동 수단임"',
   '"킥보드충 극혐"',
   '"제발 어린아이  앞에 태우고 휘청휘청 인도로 다니지 좀 마라."',
   '"세금 낭비의 대표사례.  박원숭이 얘는 진짜 최악의 행정 쓰레기."',
   '"공유킥보드는 악 그 자체  죽고싶어 안달난 자해공갈단 같다  무개념 방치  무단횡단  자동차 전용도로   차량운전자는 무조건 가해자되는게 맞는거냐?   어디서 쓰레기를 만들어서"',
   '"킥보드좀 없애자 길거리에 가장큰 장애물이자 흉물이다. 중앙버스 정류장에 저걸 세워놓는 쓰레기들도 많고 술먹고 타다가 애들한테 부딪히는 놈들도 너무많다. 제발좀 없애라."']],
 [[]],
 [[]],
 [['"생각없는 애들이야 그렇다치고, 부모들은 진짜 양심이라고는 0도없네ㅋㅋㅋ  16세이하는 면허없이 전동킥보드 타면 불법인것도 모자라서 2명이서 탑승해도 불법이고. 저렇게 고라니처럼 튀어나왔는데 어떻게 사고를 피할수있음? 부모들도 분명히 운전하는 사람들일건데, 입장바꿔 생각해보면 오히려 운전자한테 죄송하다해도 모자랄판에 합의금ㅋㅋㅋ 뻔뻔시렵다진짜ㅋㅋ"',
   '"차도 보고 바로멈췄네 가만히 있던차에 킥보드가 와서 박은건데 청소년탑승안되고 두명안되는데 누굴보고 가해자란건지 치료비 달라는 부모는 머지..."',
   '"이게 만약 운전자 과실이라면 앞으로 킥보드 타고 차에 돌진하는 아이들 엄청 늘어날거임....억울한 사람 만들지 말라고 법이 있는건데 이런 개법이 어딨냐?"',
   '"참으로 좋은 법 만들어 놨다. 잘못한 넘은 어리다고 모든게 면죄가 되나? 애들은 무면허에 두명 탑승 과속 할거는 다 했네~~~ 이게 국민을 위한 법이라고 만든사람 어디 갔노? 답변 해 봐라

In [8]:
daum_dict['comments']

[[],
 [],
 ['가해자가 촉법소년이라고 피해자가 되냐\n어째서 그런해석의 나오냐\n죄를 지어놓고 촉법소년이라고 피해자가 된다니\n단지 죄값을 안받는다는거지 피해자가 되는건 아니지\n경찰이 해석을 이상하게하네',
  '촉법이라 가해자라고???? 뭔 댕댕이 소리래?\n청소년이 칼들고 지르면, 칼 맞은 사람이 가해자냐?\n잘 못 했으면, 촉법이고 아니고를 떠나서 가해자가 가해자지.....\n촉법에 따라 죄를 사면 받은 것으 별개의 문제고....\n\n촉법을 없애라... 아니면, 촉법을 적용할 수 있는 사례를 구체적으로 만들어 제한하던지...',
  '전동킥보드 같은데.. 미성년자.. 촉법소년이면.. 당연히 무면허겠지.. 그럼 아예 타는 것조차 불법인데 블박 차량이 가해자냐?',
  '12, 10세인 아이들이 전동킥보드를 탄게 문제의 원인아닌가?   너무 어려서 주의운전이나 안전운전이란 것도 모르는 촉법소년,소녀들이  이렇게  생각없이 와서 부딪힌걸, \n그게 어째서 운전자 잘못이라고, 치료비를 물어달라고 요구하는 것인지 알 수 없다.\n아이들의 부모도 운전을 할텐데, 이건 부모의 부주의탓 아닌가?',
  '촉법소년법\n\n부모가 돈벌고 싶으면 어린자식 시켜서 차에 들이 받으라고 하면 되는 법. 완전 자해공갈 위한 법이네',
  '도대체 도로교통법에 상식이란 게 있는 거냐?\n왜 일방적으로 운전자만 희생양 삼는 거냐?\n적폐 중의 적폐다.',
  '죄를 지어도 가해자가 촉법소년이면 \n피해자가 가해자가 되고 가해자가 피해자가 되는 나라라니..\n이게 나라냐.\n대한민국 에오면 한번도 경험해보지못한걸 경험하게 되는구나',
  '촉법 없애라고!\n이 그지같은 국개들아!',
  '경찰이 문제네  법을 제대로 이해하고 집행해야지',
  '교통 관련 법규는 정말 손질이 좀 필요하다고 본다.\n대부분의 차에 블랙박스가 달려 있는 지금도\n구시대적인 법규와 판례가 지배하는 건 정말 아닌 듯',
  '법좀 개정해라\n국개는 눈뜬 장님이가\n지들 밥그릇만 챙긴다고 난리고\n국

In [154]:
## 이건 트레인 셋 용(맞춤법 검사 후 교정해주는 코드)
import re
from hanspell import spell_checker # 오탈자, 띄어쓰기 교정

# ai-hub 학습 데이터 용
def comment_clean_t(c_list):
    comment = c_list.tolist()
#    p = re.compile("\d+") # 숫자 삭제
    comment_list = []
  
    for i in range(len(comment)):    
        try:
            # 특정 특수문자 삭제
            sent = re.sub("\&|\'|\"","",comment[i])
            
            # 맞춤법 검사 및 수정
            spelled_sent = spell_checker.check(sent)
            hanspell_sent = spelled_sent.checked

            comment_list.append(hanspell_sent)
            print(f'{i} 번째 데이터 완료')
        except:
            print(comment[i])
    return comment_list

In [ ]:
# 감정분석 트레인 셋 불러와서 전처리
# aihub : 한국어 단발성 대화 데이터

In [141]:
chatbot_data = pd.read_excel('korean_short_talk.xlsx')

In [150]:
chatbot_data['Sentence']

0                                 언니 동생으로 부르는게 맞는 일인가요..??
1                                             그냥 내 느낌일뿐겠지?
2                                           아직너무초기라서 그런거죠?
3                                            유치원버스 사고 낫다던데
4                                              근데 원래이런거맞나요
                               ...                        
38589                 솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..
38590                                          재미가 없으니 망하지
38591    공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...
38592                 코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ
38593                 와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요
Name: Sentence, Length: 38594, dtype: object

In [155]:
sentence = comment_clean_t(chatbot_data['Sentence'])

0 번째 데이터 완료
1 번째 데이터 완료
2 번째 데이터 완료
3 번째 데이터 완료
4 번째 데이터 완료
5 번째 데이터 완료
6 번째 데이터 완료
7 번째 데이터 완료
8 번째 데이터 완료
9 번째 데이터 완료
10 번째 데이터 완료
11 번째 데이터 완료
12 번째 데이터 완료
13 번째 데이터 완료
14 번째 데이터 완료
15 번째 데이터 완료
16 번째 데이터 완료
17 번째 데이터 완료
18 번째 데이터 완료
19 번째 데이터 완료
20 번째 데이터 완료
21 번째 데이터 완료
22 번째 데이터 완료
23 번째 데이터 완료
24 번째 데이터 완료
25 번째 데이터 완료
26 번째 데이터 완료
27 번째 데이터 완료
28 번째 데이터 완료
29 번째 데이터 완료
30 번째 데이터 완료
31 번째 데이터 완료
32 번째 데이터 완료
33 번째 데이터 완료
34 번째 데이터 완료
35 번째 데이터 완료
36 번째 데이터 완료
37 번째 데이터 완료
38 번째 데이터 완료
39 번째 데이터 완료
40 번째 데이터 완료
41 번째 데이터 완료
42 번째 데이터 완료
43 번째 데이터 완료
44 번째 데이터 완료
45 번째 데이터 완료
46 번째 데이터 완료
47 번째 데이터 완료
48 번째 데이터 완료
49 번째 데이터 완료
50 번째 데이터 완료
51 번째 데이터 완료
52 번째 데이터 완료
53 번째 데이터 완료
54 번째 데이터 완료
55 번째 데이터 완료
56 번째 데이터 완료
57 번째 데이터 완료
58 번째 데이터 완료
59 번째 데이터 완료
60 번째 데이터 완료
61 번째 데이터 완료
62 번째 데이터 완료
63 번째 데이터 완료
64 번째 데이터 완료
65 번째 데이터 완료
66 번째 데이터 완료
67 번째 데이터 완료
68 번째 데이터 완료
69 번째 데이터 완료
70 번째 데이터 완료
71 번째 데이터 완료
72 번째 데이터 완료
73 번째 데이터 완료
74 번째 데이터 완료
75 번째 데이터 완료
76 번째 데이터 완료
77 번째 데이터

In [157]:
sentence

['언니 동생으로 부르는 게 맞는 일인가요..??',
 '그냥 내 느낌일 뿐이겠지?',
 '아직 너무 초기라서 그런 거죠?',
 '유치원 버스 사고 낫다던데',
 '근데 원래 이런 거 맞나요',
 ' 남자친구가 떠날까 봐요',
 '이거 했는데 허리가 아플 수도 있나요? ;;',
 '내가 불안해서 꾸는 걸까..',
 ' 일주일도 안 남았다... ᅲᅲ',
 '약은 최대한 안 먹으려고 하는 데 좋은 음 시 있나요? 0',
 ' 구직활동하면서 남는 시간은 뭘로 활용해야 되지..',
 '괜찮은 분 같아서 괜히 조급해지네요 ㅜ',
 ' 제가 스타일을 바꾸면 더 어색하게만 변할 것 같아서 ㅠ0',
 '아내 있으면 여자 있는 술집 가면 법 적으로 문제 있나요?',
 ' 한 소리들을 지.... 도와주세요',
 '도와주실 분 없나요..',
 '그냥 잡다한 거 말고 학업에 관련된 거 듣는 게 더 좋은가요?',
 ' 가격은 반값.... 이거 질이 다른 건가요??',
 '마지막에 말씀드렸던 대로 글은 이제 내릴게요의 도치 않게 실시간 베스트 글에 올라가서 남자친구가 볼 거 같아서요 ㅠㅠ',
 '화장품 회사다 보니, 젊은 여자들이 많은 거 같은데 걱정이네요..',
 ' 몸무게 1킬로 찌는 건 아니겠죠?',
 'o자 다리 많이 심한가요?',
 ' 안 하는 게 나을까요??',
 '그 정도로 더러운가요.........',
 ' 보통 가진 콩도 이렇게 오래가나요?',
 '여자가 술 취해서 먼저 전화하는 거 짜증 나요???',
 '걱정이 큽니다',
 '이거 무슨 증상이죠??ㅜㅜ',
 ' 다름 아니라 제 동생 걱정입니다',
 '아무래도 무리겠죠?',
 '애인으로써 정말 걱정됩니다',
 '이러다 일주일 전에 나오는 거 아닌가 몰라요',
 '경고 먹을까 고민 0',
 '극도로 예민하고 불안한 상태에요',
 ' 못 받으면 어떡하죠',
 ' 안검하수도 어울릴까요?',
 '근데 그게 사마귀 일수 있데요.',
 '이제 곧 내년이면 22살인 여대생입니다... 요즘 고민이 있는데요...',
 ' 안 하는 게 

In [158]:
data = pd.concat([pd.DataFrame({'Sentence' : sentence}), chatbot_data["Emotion"]], axis = 1)

In [160]:
# train set 저장
# 이 데이터를 코랩에서 이용해 학습
with open(path+'train_hanspell_new.pickle', 'wb') as f:
    pickle.dump(data, f)

In [169]:
## 이건 네이버, 다음 댓글용(맞춤법 검사 후 교정해주는 코드)
import re
from hanspell import spell_checker # 오탈자, 띄어쓰기 교정

def comment_clean(dic):
    comment = dic['comments']
    p = re.compile("\&|\'|\"") # 특수문자, 숫자 삭제
    comment_list = []
    
    for i in range(len(comment)):    
        try:
            comment_j = []
            
            if type(comment[i][0]) == list:
                cm = sum(comment[i], [])
            else:
                cm = comment[i]
        
            for j in range(len(cm)):
                
                # 1. 특수문자, 숫자 삭제
                sent = p.sub(" ", cm[j]) 

                spelled_sent = spell_checker.check(sent)
                hanspell_sent = spelled_sent.checked
                
                comment_j.append(hanspell_sent)
                
        except:
            pass

        comment_list.append(comment_j)
        print(f'{i} 번째 데이터 완료')
    return comment_list

In [170]:
daum = comment_clean(daum_dict)
len(daum)

0 번째 데이터 완료
1 번째 데이터 완료
2 번째 데이터 완료
3 번째 데이터 완료
4 번째 데이터 완료
5 번째 데이터 완료
6 번째 데이터 완료
7 번째 데이터 완료
8 번째 데이터 완료
9 번째 데이터 완료
10 번째 데이터 완료
11 번째 데이터 완료
12 번째 데이터 완료
13 번째 데이터 완료
14 번째 데이터 완료
15 번째 데이터 완료
16 번째 데이터 완료
17 번째 데이터 완료
18 번째 데이터 완료
19 번째 데이터 완료
20 번째 데이터 완료
21 번째 데이터 완료
22 번째 데이터 완료
23 번째 데이터 완료
24 번째 데이터 완료
25 번째 데이터 완료
26 번째 데이터 완료
27 번째 데이터 완료
28 번째 데이터 완료
29 번째 데이터 완료
30 번째 데이터 완료
31 번째 데이터 완료
32 번째 데이터 완료
33 번째 데이터 완료
34 번째 데이터 완료
35 번째 데이터 완료
36 번째 데이터 완료
37 번째 데이터 완료
38 번째 데이터 완료
39 번째 데이터 완료
40 번째 데이터 완료
41 번째 데이터 완료
42 번째 데이터 완료
43 번째 데이터 완료
44 번째 데이터 완료
45 번째 데이터 완료
46 번째 데이터 완료
47 번째 데이터 완료
48 번째 데이터 완료
49 번째 데이터 완료
50 번째 데이터 완료
51 번째 데이터 완료
52 번째 데이터 완료
53 번째 데이터 완료
54 번째 데이터 완료
55 번째 데이터 완료
56 번째 데이터 완료
57 번째 데이터 완료
58 번째 데이터 완료
59 번째 데이터 완료
60 번째 데이터 완료
61 번째 데이터 완료
62 번째 데이터 완료
63 번째 데이터 완료
64 번째 데이터 완료
65 번째 데이터 완료
66 번째 데이터 완료
67 번째 데이터 완료
68 번째 데이터 완료
69 번째 데이터 완료
70 번째 데이터 완료
71 번째 데이터 완료
72 번째 데이터 완료
73 번째 데이터 완료
74 번째 데이터 완료
75 번째 데이터 완료
76 번째 데이터 완료
77 번째 데이터

276

In [172]:
naver = comment_clean(naver_dict)
len(naver)

0 번째 데이터 완료
1 번째 데이터 완료
2 번째 데이터 완료
3 번째 데이터 완료
4 번째 데이터 완료
5 번째 데이터 완료
6 번째 데이터 완료
7 번째 데이터 완료
8 번째 데이터 완료
9 번째 데이터 완료
10 번째 데이터 완료
11 번째 데이터 완료
12 번째 데이터 완료
13 번째 데이터 완료
14 번째 데이터 완료
15 번째 데이터 완료
16 번째 데이터 완료
17 번째 데이터 완료
18 번째 데이터 완료
19 번째 데이터 완료
20 번째 데이터 완료
21 번째 데이터 완료
22 번째 데이터 완료
23 번째 데이터 완료
24 번째 데이터 완료
25 번째 데이터 완료
26 번째 데이터 완료
27 번째 데이터 완료
28 번째 데이터 완료
29 번째 데이터 완료
30 번째 데이터 완료
31 번째 데이터 완료
32 번째 데이터 완료
33 번째 데이터 완료
34 번째 데이터 완료
35 번째 데이터 완료
36 번째 데이터 완료
37 번째 데이터 완료
38 번째 데이터 완료
39 번째 데이터 완료
40 번째 데이터 완료
41 번째 데이터 완료
42 번째 데이터 완료
43 번째 데이터 완료
44 번째 데이터 완료
45 번째 데이터 완료
46 번째 데이터 완료
47 번째 데이터 완료
48 번째 데이터 완료
49 번째 데이터 완료
50 번째 데이터 완료
51 번째 데이터 완료
52 번째 데이터 완료
53 번째 데이터 완료
54 번째 데이터 완료
55 번째 데이터 완료
56 번째 데이터 완료
57 번째 데이터 완료
58 번째 데이터 완료
59 번째 데이터 완료
60 번째 데이터 완료
61 번째 데이터 완료
62 번째 데이터 완료
63 번째 데이터 완료
64 번째 데이터 완료
65 번째 데이터 완료
66 번째 데이터 완료
67 번째 데이터 완료
68 번째 데이터 완료
69 번째 데이터 완료
70 번째 데이터 완료
71 번째 데이터 완료
72 번째 데이터 완료
73 번째 데이터 완료
74 번째 데이터 완료
75 번째 데이터 완료
76 번째 데이터 완료
77 번째 데이터

1896

In [173]:
# # 저장
# with open(path+'naver_comment_new.pickle', 'wb') as f:
#     pickle.dump(naver, f)

# with open(path+'daum_comment_new.pickle', 'wb') as f:
#     pickle.dump(daum, f)

In [4]:
# # 열기
path = 'C:/Users/82103/text_data/'

with open(path+'naver_comment_new.pickle', 'rb') as f:
    naver_comment = pickle.load(f)
    
with open(path+'daum_comment_new.pickle', 'rb') as f:
    daum_comment = pickle.load(f)

In [156]:
print(len(naver_comment), len(daum_comment))

1896 276


In [157]:
# 댓글 관련 2차 필터링 함수 생성(기사마다 리스트로)
def comment_clean(comment_list):
    
    clean_list = []
    p = re.compile("\W+|\d+")
    tokenizer = Okt()
    
    for i in range(len(comment_list)):
        try:
            comment = comment_list[i]
            comment_j = []
            
            for j in range(len(comment)):
                
                # 1. 특수문자, 숫자 삭제
                sent = p.sub(" ", comment[j]) 
                
                # 2. konlpy에서 형태소 분석을 통해 형태소로 분리
                tokens = []
                token = tokenizer.morphs(sent, stem = True) # 모든 형태소 처리(morphs) ,명사만 뽑기(nouns)
                token = [t for t in token if t not in stopwords] # 불용어 필터
                token = [t for t in token if len(t) > 1] # 글자 하나는 제거
                tokens.append(' '.join(token))

                # 3. L tokenizer 이용해서 토큰 리스트 생성
                clean = []
                words = l_tokenizer.tokenize(tokens[0], flatten = False)
                words = [x for x, y in words if len(x) > 1] # 앞에꺼(L)만 출력, 두 글자 이상만
                words = [x for x in words if x not in stopwords] # 불용어 제거해주기
                clean.append(' '.join(words))
                comment_j.append(clean[0])
        except:
            pass
        clean_list.append(comment_j)
        
    return(clean_list)

In [158]:
naver_cm_clean = comment_clean(naver_comment)
daum_cm_clean = comment_clean(daum_comment)

In [58]:
naver_cm_clean

[['모르다 공유 보드 없애다 맞다 무면허 헬멧 짐승 하다 경우 많다 상당하다 위험하다',
  '공유 보드 안전 안전 생각 보다 요금 가끔 신호 잘못 걸리 그냥 편안하 택시 맞다 듯이 생각 들다 적절 애매 사이 여러 생각 하다 만들 이동 수단',
  '보드 극협',
  '제발 어리다 태우다 휘청휘청 인도 다니다 마르다',
  '세금 낭비 대표 사례 원숭이 진짜 최악 행정 쓰레기',
  '공유 보드 자체 죽다 싶다 안달 자해 공갈 개념 방치 무단 횡단 자동차 전용 도로 차량 운전자 무조건 가해자 되다 맞다 어디서 쓰레기 만들',
  '보드 없애다 길거리 가장 크다 장애 이자 흉물 이다 중앙 버스 정류장 저걸 세우다 쓰레기 많다 먹다 다가 한테 부딪히다 너무 많다 제발 없애다'],
 [],
 [],
 ['생각 없다 이야 그렇다 치고 부모 진짜 양심 이라고 없다 ㅋㅋ 세이 하다 면허 없이 전동 보드 불법 모자라다 탑승 하다 불법 이고 저렇게 고라니 처럼 튀어나 어떻 사고 하다 부모 분명하다 운전 하다 사람 들이다 건데 입장 바꾸다 생각 해보다 운전자 한테 죄송하다 해도 모자라다 합의 ㅋㅋ 뻔뻔 시리 진짜 ᄏᄏ',
  '차도 보고 멈추 가만히 보드 오다 박다 건데 청소년 탑승 안되다 안되다 누굴 보고 가해자 건지다 치료 달라 부모',
  '운전자 과실 보드 타고 돌진 하다 엄청 늘어 거임 억울하다 사람 만들 건데 이렇다 개법 어딨다',
  '참으로 좋다 만들 놨다 잘못 넘다 어리다 모든 면죄 되다 무면허 탑승 과속 하다 하다 국민 이라고 만들 사람 가다 답변 하다 보다',
  '이렇다 썩다 빠지 경찰 조직 공소권 수사 주다 문재인 민주당 정권 개돼지 들이다',
  '정지 하다 보드 속도 방향 제어 하고 들이다 받다 경찰 편하다 처리 하다 하다 저렇게 처리 하다 경찰 하다 개월 감봉 하다 제대로 하다',
  '너무 보다 속도 정말 저속 인데 저리 갑자기 튀다 보드 피다 차갑다 우주선 인가요 하늘 날아오르 피해 하나요 촉법 따다 근본 먼저 따지다',
  '자식 두다 부모

In [67]:
daum_cm_clean

[[],
 [],
 ['가해자 촉법소년 이라고 피해 되다 그렇다 해석 나오 지다 촉법소년 이라고 피해 되다 죗값 받다 거지 피해 되다 경찰 해석 하다',
  '촉법 이라 가해자 댕댕 소리 청소년 들다 지르다 맞다 사람 가해자 자다 하다 촉법 이고 떠나 가해자 가해 자지 촉법 따르 사면 받다 별개 문제 촉법 없애다 촉법 적용 하다 사례 구체 만들 제한 하다',
  '전동 보드 미성년 촉법소년 당연하다 무면허 겠다 그렇다 아예 불법 인데 크다 차량 가해자',
  '세인 전동 보드 문제 원인 너무 어리다 주의 운전 이나 안전 운전 이란 모르다 촉법소년 소녀 이렇게 생각 없이 오다 부딪히다 그게 운전자 잘못 이라고 치료 물어 달라 요구 하다 인지 없다 부모 운전 하다 텐데 이건 부모 부주의',
  '촉법소년 부모 벌다 싶다 어리다 자식 시키다 들이다 받다 하다 되다 완전 자해 공갈 이네',
  '도대체 도로교통법 상식 이란 일방 운전자 희생 삼다 적폐 적폐',
  '어도 가해자 촉법소년 피해 가해자 되다 가해자 피해 되다 나라 라니 나라 대한민국 에이 하다 경험 해보다 경험 하다 되다',
  '촉법 없애다 거지 국가',
  '경찰 문제 제대로 이해 하고 집행 하다',
  '교통 관련 법규 정말 손질 필요 보다 대부분 블랙 달다 지금 구시대 법규 판례 지배 하다 정말',
  '개정 하다 국가 눈뜨다 장님 밥그릇 챙기 난리 국민 원성 들리다',
  '소년법 폐지 하다 쓰레기 청소 이다',
  '피하 려면 보드 보다 순간 브레이크 밟다 동시 후진 기어 넣다 지체 없이 액셀 밟다 하다 가능 없다 보이',
  '그냥 아무 모르다 가해자 부모 억지 써다 억지 씌우 커지',
  '현실 바뀌다 적용 되어 하다 입법기관 하다 국회의원 역할 해주다 바라',
  '미래 사기꾼 양성 하다 촉법',
  '저걸 어떻 피하 나라 자다 돌아',
  '민사 소송 가야',
  '',
  '쓰레기 이란 가지 장난 치다 차다 한탄 스럽다',
  '민주당 바꾸다',
  '촉법소년',
  '경찰 잘못 하다

In [159]:
# ## 정제한 댓글 최종 데이터
# with open(path+'naver_cm_clean.pickle', 'wb') as f:
#     pickle.dump(naver_cm_clean, f)

# with open(path+'daum_cm_clean.pickle', 'wb') as f:
#     pickle.dump(daum_cm_clean, f)